<a href="https://colab.research.google.com/github/jungeun919/Pytorch_study/blob/main/Pytorch_Tutorial/1.6Autograd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Autograd

- Tensor의 모든 연산에 대한 자동 미분을 제공
- .requires_grad=True: Tensor에서 이루어진 모든 연산을 추적, 기록
- .backward(): 모든 변화도(gradient)를 자동 계산
- Tensor의 변화도는 .grad 속성에 누적됨
- .detach(): Tensor가 기록을 추적하는 것을 중단
- with torch.no_grad(): : 기록을 추적하는 것을 방지<br>
→ 변화도(gradient)는 필요없지만 requires_grad=True가 설정되어 학습 가능한 매개변수를 갖는 모델을 평가할 때 유용함

In [13]:
import torch

x = torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [14]:
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [15]:
# y는 연산의 결과로 생성된 것이므로 grad_fn을 가짐
print(y.grad_fn)

In [16]:
z = y * y * 3
out = z.mean()
print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


In [17]:
# .requires_grad_()는 기존 Tensor의 requires_grad 값을 바꿔치기하여 변경
# 입력값이 지정되지 않으면 기본값은 False
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)

b = (a * a).sum()
print(b.grad_fn)

False
True


# 변화도 (Gradient)

In [18]:
# 위에서 out(tensor(27., grad_fn=<MeanBackward0>))은 하나의 스칼라 값만 가지고 있기 때문에
# out.backward() == out.backward(torch.tensor(1.))
out.backward()

In [19]:
# 변화도 d(out)/dx
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


- 야코비안 행렬
- 벡터 함수 y = f(x)에서 x에 다한 y의 변화도

In [20]:
# 벡터-야코비안 곱의 예제
x = torch.randn(3, requires_grad=True)

y = x * 2
while y.data.norm() < 1000: # norm을 사용하여 점 사이의 거리를 계산
    y = y * 2

print(y)

tensor([ -711.1924, -1250.8550,   348.3162], grad_fn=<MulBackward0>)


* 위에서 y는 더 이상 스칼라 값이 아님
* torch.autograd는 전체 야코비안을 직접 계산할 수 없음
* 벡터-야코비안 곱은 backward에 해당 벡터를 인자로 제공하여 얻음

In [21]:
v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(v)

print(x.grad)

tensor([5.1200e+01, 5.1200e+02, 5.1200e-02])


In [22]:
print(x.requires_grad)
print((x ** 2).requires_grad)

# .requires_grad=True인 Tensor의 연산 기록을 추적하는 것을 멈추도록 함
with torch.no_grad():
    print((x **2).requires_grad)

True
True
False


In [23]:
# .detach()를 호출하여 내용물은 같지만 require_grad가 다른 새로운 Tensor를 가져옴
print(x.requires_grad)
y = x.detach()
print(y.requires_grad)
print(x.eq(y).all())

True
False
tensor(True)
